<a href="https://colab.research.google.com/github/Masum06/GenderDebiasing/blob/development2/Gender_Encrypter_Decrypter_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Latest

https://www.researchgate.net/publication/337918817_What_You_See_Is_What_You_Learn_Mitigating_Gender_Biases_from_Natural_Language_Processing_Context_through_Gender_Encrypted_Training 

# Gender Encryption

### Installation

In [1]:
import spacy
spacy.__version__

'2.2.4'

https://github.com/zalandoresearch/flair

https://github.com/huggingface/neuralcoref

In [2]:
### Restart Runtime after spacy

!pip uninstall torchvision --yes
!pip install torchvision==0.4.0
!pip install flair==0.4.2
!pip uninstall spacy --yes
!pip install -U spacy==2.1.0

Uninstalling torchvision-0.7.0+cu101:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/torchvision-0.7.0+cu101.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torchvision/*
Proceed (y/n)? y
  Successfully uninstalled torchvision-0.7.0+cu101
     |████████████████████████████████| 8.8MB 4.9MB/s 
     |████████████████████████████████| 748.9MB 13kB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
     |████████████████████████████████| 143kB 6.5MB/s 
     |████████████████████████████████| 798kB 22.1MB/s 
     |████████████████████████████████| 133kB 51.0MB/s 
     |████████████████████████████████| 1.1MB 30.3MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116677 sha256=6316e02130d5932a404bba7209049e34660136f35cfc44074d8457913ca8dc7d
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for

**Restart Runtime**

In [3]:
##!python -m spacy download en # SMALL VERSION OF SPACY, FAST DOWNLOAD
##!python -m spacy download en_core_web_md # MEDIUM VERSION OF SPACY
!python -m spacy download en_core_web_lg # LARGE VERSION OF SPACY, SLOW DOWNLOAD
!pip install neuralcoref --no-binary neuralcoref

     |████████████████████████████████| 27.7MB 111kB/s 
     |████████████████████████████████| 2.1MB 46.5MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
     |████████████████████████████████| 3.2MB 46.1MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.0 which is incompatible.
  Found existing installation: preshed 3.0.2
    Uninstalling preshed-3.0.2:
      Successfully uninstalled preshed-3.0.2
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0


     |████████████████████████████████| 826.9MB 44.8MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255078 sha256=d435bc78d501baa963bf968f485d56bca0d716be74103c8e936950babe57a337
  Stored in directory: /tmp/pip-ephem-wheel-cache-7czlhhmt/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
     |████████████████████████████████| 378kB 4.5MB/s 
Skipping wheel build for neuralcoref, due to binaries being disabled for it.
    Running setup.py install for neuralcoref ... done


In [1]:
import torch
torch.cuda.get_device_name(0)

'Tesla T4'

In [2]:
!nvidia-smi

Sat Aug 29 18:38:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Creating Language Resources

In [3]:
male_pronouns   = ["he",  "him", "his$", "his", "himself"]
female_pronouns = ["she", "her", "her$", "hers", "herself"]
neutral_pronouns= ["zie", "zim", "zir", "zis", "zieself"]
merged_pronouns = ["he/she", "him/her", "his/her", "his/hers", "himself/herself"]

In [4]:
gender_pronouns_dict = {}
gender_honorific_dict = {}

In [5]:
for (g1,g2,g3,g4) in zip(male_pronouns, female_pronouns, neutral_pronouns, merged_pronouns):
    element = {"male": g1.replace("$",""), "female":g2.replace("$",""), "neutral":g3, "merged":g4}
    gender_pronouns_dict[g1] = gender_pronouns_dict[g2] = gender_pronouns_dict[g3] = gender_pronouns_dict[g4] =element

In [6]:
male_hons   =  ["Mr.", "Mr", "Md.", "Md", "Sir", "Lord", "Mister"]
female_hons =  ["Ms.", "Ms", "Mst.", "Mst", "Madam", "Lady", "Miss"]
neutral_hons = ["Mx.", "Mx", "Mx.", "Mx", "Sir/Madam", "Lord/Lady", "Mister/Miss"]
married_hons = ["Mrs.", "Mrs", "Mst.", "Mst", "Madam", "Lady", "Mis'ess"]
merged_hons =  ["Mr./Ms.", "Mr/Ms", "Md./Mst.", "Md/Mst", "Sir/Madam", "Lord/Lady", "Mister/Miss"]

In [7]:
for (h1,h2,h3,h4,h5) in zip(male_hons, female_hons, neutral_hons, married_hons, merged_hons):
    element = {"male": h1, "female":h2, "neutral":h3, "married_fem":h4, "merged":h5}
    gender_honorific_dict[h1] = gender_honorific_dict[h2] = gender_honorific_dict[h3] = \
    gender_honorific_dict[h4] = gender_honorific_dict[h5] = element

### Imports

In [8]:
import re

In [9]:
from flair.data import Sentence
from flair.models import SequenceTagger
#from segtok.segmenter import split_single

In [10]:
import flair
assert flair.__version__=='0.4.2'

In [11]:
from keras.models import load_model

In [12]:
import spacy
import neuralcoref

100%|██████████| 40155833/40155833 [00:01<00:00, 26353388.52B/s]


In [13]:
import keras
keras.__version__

'2.4.3'

In [14]:
import torch
torch.__version__

'1.2.0'

### Loading Essentials

In [15]:
## Only do this on colab
!git clone --single-branch --branch development2 https://github.com/Masum06/GenderDebiasing.git

Cloning into 'GenderDebiasing'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 160 (delta 19), reused 9 (delta 4), pack-reused 127
Receiving objects: 100% (160/160), 43.27 MiB | 21.60 MiB/s, done.
Resolving deltas: 100% (78/78), done.
Checking out files: 100% (47/47), done.


In [17]:
ls

GenderDebiasing/  sample_data/


In [18]:
import json

char2idx = 'GenderDebiasing/resources/char2idx.json'
idx2char = 'GenderDebiasing/resources/idx2char.json'
with open(char2idx, 'r') as fp:
    char2idx = json.load(fp)
    
with open(idx2char, 'r') as fp:
    idx2char = json.load(fp)

In [19]:
model_name = 'GenderDebiasing/resources/char_rnn_hsc_model_0.h5'
model = load_model(model_name)

2020-08-29 18:39:44,096 Layer lstm_1 will not use cuDNN kernel since it doesn't meet the cuDNN kernel criteria. It will use generic GPU kernel as fallback when running on GPU


In [20]:
#nlp = spacy.load('en_core_web_lg') # en_core_web_sm #en_core_web_md
import en_core_web_lg
nlp = en_core_web_lg.load()
neuralcoref.add_to_pipe(nlp)

In [21]:
tagger_ner = SequenceTagger.load('ner')
tagger_pos = SequenceTagger.load('pos')

OSError: ignored

### Necessary Methods

Name2Gender

In [ ]:
import numpy as np
from keras.preprocessing import sequence

# Converts a name into vector
def name2vectorTest(name):
    name = name.lower()
    new_name = ""
    for char in name:
      if char in char2idx:
        new_name += char
    chars = list(new_name)
    vector = [ char2idx[c] for c in chars ]
    return np.array(vector)

# Converts names to fixed size tensor
def names2tensorTest(names, maxlen=25):
    namelist = [name2vectorTest(name) for name in names]
    return sequence.pad_sequences(np.array(namelist), maxlen=maxlen)  # root of all troubles

def name2gender(name):
  result = model.predict_classes(np.array(names2tensorTest([name.lower()])))[0][0]
  if result:
    return "male"
  else:
    return "female"
  
def isMale(name):
  result = model.predict_classes(np.array(names2tensorTest([name.lower()])))[0][0]
  return result

Storing Names in Dictionary

In [ ]:
def store(name, name_found, Name2Key, Key2Name, num_keys): 
    
    if name_found not in Name2Key:
        #global num_keys
        num_keys+=1
        key = "PER_"+str(num_keys)
        #gender = name2gender(name_found)
        alias = None
        element = {"name": name, "key": key, "gender":None, "alias":alias, "is_alias": False, "alias_to": None}
        Name2Key[name_found] = element
        Key2Name[key] = {"name": name, "gender": None, "alias": alias}
    
    if name not in Name2Key:
        element_alias = {"name": name, "is_alias": True, "alias_to": name_found}
        Name2Key[name] = element_alias
        Name2Key[name_found]["alias"] = name
        
    return Name2Key[name_found]["key"], num_keys

# Unit Gender Encryption

In [ ]:
def gender_encrypt(s):
  s = ' '.join(s.split())
  doc = nlp(s)
  tokenized_text = ' '.join([token.text for token in doc])
  oracle = []
  coref2name = {}

  # POS TAG
  sent = Sentence(tokenized_text)
  tagger_ner.predict(sent)
  tagger_pos.predict(sent)
  tagged_list = sent.to_tagged_string().split()
  tokens_pos = []
  pos = []
  count = 0
  for i in range(0,len(tagged_list),2):
      tokens_pos.append(tagged_list[i])
      count = count+1
      pos.append(tagged_list[i+1])
      
      if tagged_list[i].lower() in male_pronouns+female_pronouns:
          oracle.append(2)
      elif tagged_list[i+1] in ['<B-PER/NNP>', '<I-PER/NNP>', '<E-PER/NNP>', '<S-PER/NNP>']:
          oracle.append(4)
      else:
          oracle.append(0)

  # COREFERENCE RESOLUTION 
  if len(doc)!=len(tokens_pos):
    tokens_doc = [token.text for token in doc]
    print("doc:", tokens_doc)
    print("pos:", tokens_pos)
    return None, None
  
  coref_stack = []
  name_stack = []
  for i in range(len(doc)):
      token = doc[i]
      if token._.in_coref:
          coref_stack.append(tokens_pos[i])
          if oracle[i] == 4:
              name_stack.append(tokens_pos[i])
          oracle[i] += 1
      else:
          if len(name_stack) > 0:
              name = ' '.join(name_stack)
              coref = ' '.join(coref_stack)
              #name2coref[name] = coref
              coref2name[coref] = name
              name_stack.clear()
          coref_stack.clear()

  # IF THE SENTENCE DOES NOT END WITH A PERIOD OR SPECIAL CHARACTER
  if len(name_stack) > 0:
      name = ' '.join(name_stack)
      name2coref[name] = ' '.join(coref_stack)
      name_stack.clear()
  coref_stack.clear()

  Name2Key = {}
  Key2Name = {}
  encrypted = []
  num_keys = 0
  i = 0
  while i<len(tokens_pos): 
      #print("Oracle ", i, tokens_pos[i])
      if oracle[i] == 2:
          pronoun = tokens_pos[i].lower()
          if pos[i] == '<PRP$>':
              pronoun+="$"
          encrypted.append(gender_pronouns_dict[pronoun]["merged"])
      elif oracle[i] == 3:
          coref = doc[i]._.coref_clusters[0][0].text
          pronoun = tokens_pos[i].lower()
          if pos[i] == '<PRP$>':
              pronoun+="$"
          if coref in coref2name:
            name_found = coref2name[coref]
            key = Name2Key[name_found]["key"]
            if pronoun in male_pronouns:
              Name2Key[name_found]["gender"] = "male"
            elif pronoun in female_pronouns:
              Name2Key[name_found]["gender"] = "female"
            else:
              Name2Key[name_found]["gender"] = "<|unk|>"
            encrypted.append("<|coref|>")
            encrypted.append(gender_pronouns_dict[pronoun]["merged"])
            encrypted.append(key)
          else:
            encrypted.append(gender_pronouns_dict[pronoun]["merged"])
          
      elif oracle[i] in [4,5]:
          if i > 0 and tokens_pos[i-1] in gender_honorific_dict:
            hons = encrypted.pop()
            if hons in male_hons:
              gender = "male"
            elif hons in female_hons+married_hons:
              gender = "female"
            else:
              gender = "<|unk|>"
            encrypted.append("<|hons|>")
            encrypted.append(gender_honorific_dict[hons]["merged"])
          if pos[i] == '<S-PER/NNP>':
              name = tokens_pos[i]
          elif pos[i] == '<B-PER/NNP>':
              name = ""
              while True:
                  #print(i, oracle[i])
                  name += tokens_pos[i]
                  if pos[i] == '<E-PER/NNP>':
                      break
                  name += " "
                  i+=1
          
          if oracle[i] == 4:
              key, num_keys = store(name, name, Name2Key, Key2Name, num_keys)
              encrypted.append(key)
          else:
              coref = doc[i]._.coref_clusters[0][0].text
              name_found = coref2name[coref]
              if name == name_found:
                  key, num_keys = store(name, name_found, Name2Key, Key2Name, num_keys)
                  encrypted.append(key)
              else:
                  key, num_keys = store(name, name_found, Name2Key, Key2Name, num_keys)
                  encrypted.append("<|alias|>")
                  encrypted.append(key)
          #Name2Key[key]["gender"] = gender
      else:
        encrypted.append(tokens_pos[i])
      i+=1
      
  encrypted_text = ' '.join(encrypted)

  return tokenized_text, Key2Name, encrypted_text

# Gender Decryption

In [ ]:
def gender_decrypt(encrypted_text, Key2Name):
  encrypted = encrypted_text.split()
  decrypted = []
  i = 0
  while i != len(encrypted):
    token = encrypted[i]
    if token in Key2Name:
      decrypted.append(Key2Name[token]["name"])
    elif token == '<|alias|>':
      i+=1
      key = encrypted[i]
      if Key2Name[key]["alias"]:
        decrypted.append(Key2Name[key]["alias"])
      else:
        decrypted.append(Key2Name[key]["name"])
    elif token == '<|coref|>':
      startOfSent = (i==0 or encrypted[i-1] == ".")
      pronoun =  encrypted[i+1]
      key = encrypted[i+2]
      gender = Key2Name[key]["gender"].lower()
      decrypted_pronoun = gender_pronouns_dict[pronoun][gender]
      decrypted_pronoun = decrypted_pronoun[0].upper()+decrypted_pronoun[1:] if startOfSent else decrypted_pronoun
      decrypted.append(decrypted_pronoun)
      i+=2
    elif token == "<|hons|>":
      hons = encrypted[i+1]
      key = encrypted[i+2] # NEED CHECK FOR HONS IN ALIAS
      gender = Key2Name[key]["gender"].lower()
      decrypted.append(gender_honorific_dict[hons][gender])
      i+=1
    elif token == "he/she":
        if i==0 or encrypted[i-1] == ".":
            decrypted.append("He/She")
        else:
            decrypted.append("he/she")
    else:
      decrypted.append(token)
    i+=1

  decrypted_text = ' '.join(decrypted)
  #decrypted_text = decrypted_text.replace(" .", ".")
  return decrypted_text

In [ ]:
key2name_changed = {'PER_1': {'name': 'Abira',
    'key': 'PER_1',
    'gender': 'Female',
    'alias': None,
    'is_alias': False,
    'alias_to': None},
  'PER_2': {'name': 'Shibir',
    'key': 'PER_2',
    'gender': 'Male',
    'alias': None,
    'is_alias': False,
    'alias_to': None}
  }

### Testing Encryption-Decryption

In [ ]:
s = "Mr. John is not a doctor. Ms. Katy is also not a nurse. John can fly, but she cannot swim."

In [ ]:
s = "Morshed Khan told newsmen that 'BNP is a massive party with huge popularity and public acceptance.' This party is now running its operations over Skype. He (Tarique) is operating from London over Skype and that too with only selected leaders. It has turned into the ‘Bangladesh Nationalist Skype Party’. This is painful. I believe the next generation must assume leadership within the party — that would be my recommendation.”"

In [ ]:
s = "Dineshwar then left behind his wife’s stabbed body and hanged himself from a tree in a field nearby."

In [ ]:
len(s.split())

18

In [ ]:
import datetime 
start = datetime.datetime.now()
text, key2name, encrypted = gender_encrypt(s)
end = datetime.datetime.now()

print(encrypted)
print(key2name)
print(end-start)

PER_1 then left behind <|coref|> his/her PER_1 wife ’s stabbed body and hanged <|coref|> himself/herself PER_1 from a tree in a field nearby .
{'PER_1': {'name': 'Dineshwar', 'gender': None, 'alias': None}}
0:00:00.307706


In [ ]:
key2name = {'PER_1': {'name': 'Dineshwar', 'gender': 'merged', 'alias': None}}

In [ ]:
decrypted = gender_decrypt(encrypted, key2name)
decrypted

'Dineshwar then left behind his/her wife ’s stabbed body and hanged himself/herself from a tree in a field nearby .'

In [ ]:
decrypted == text

True

In [ ]:
en = gender_encrypt("Mr Masum Hasan is not a doctor. Masum is an engineer.")
en

NameError: name 'gender_encrypt' is not defined

In [ ]:
isMale("Masum")

1